<h1>Importações</h1>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
import datetime
import numpy as np
mt5.initialize()
ativo ='WIN$N'
ticks = pd.DataFrame(mt5.copy_ticks_range('WIN$N',datetime.datetime(2023,10,25),datetime.datetime(2023,12,14),mt5.COPY_TICKS_ALL))
ticks['time'] =pd.to_datetime(ticks['time'],unit='s')
grupo = ticks.groupby('last').agg({'volume':'sum'})
x = tuple((grupo.loc[(grupo['volume'] == grupo['volume'].max())].reset_index()).values)
ticks
# Excluir o caso onde ele não abre encima da poc e volta, colocar um range de 10,20,30,40, analisar dia de tendência 

,time,bid,ask,last,volume,time_msc,flags,volume_real
0,2023-10-25 09:00:57,0.0,0.0,115400.0,3014,1698224457339,312,3014.0
1,2023-10-25 09:00:57,0.0,0.0,115400.0,2093,1698224457340,312,2093.0
2,2023-10-25 09:00:57,0.0,0.0,115400.0,1778,1698224457342,312,1778.0
3,2023-10-25 09:00:57,0.0,0.0,115400.0,168,1698224457343,312,168.0
4,2023-10-25 09:00:57,0.0,0.0,115400.0,1282,1698224457343,312,1282.0
...,...,...,...,...,...,...,...,...
42788399,2023-12-13 17:59:56,0.0,0.0,128020.0,1,1702490396506,312,1.0
42788400,2023-12-13 17:59:57,0.0,0.0,128010.0,2,1702490397620,312,2.0
42788401,2023-12-13 17:59:59,0.0,0.0,128020.0,1,1702490399077,312,1.0
42788402,2023-12-13 17:59:59,0.0,0.0,128020.0,1,1702490399601,312,1.0


<h1>Puxando a POC do dia</h1>

In [2]:
poc = {'Data':[],'POC':[],'Volume':[]}

dia = datetime.datetime(year=2023,month=4,day=1)
dia_1 = dia + datetime.timedelta(days=1)
for i in range(400):
    ticks = pd.DataFrame(mt5.copy_ticks_range(ativo,dia,dia_1,mt5.COPY_TICKS_ALL))
    grupo = ticks.groupby('last').agg({'volume':'sum'}) 
    y = tuple((grupo.loc[(grupo['volume'] == grupo['volume'].max())].reset_index()).values)
    if len(y)!=0:
        last = y[0][0]
        volume = y[0][1]
        poc['Data'].append(dia)
        poc['POC'].append(last)
        poc['Volume'].append(volume)
        dia = dia + datetime.timedelta(days=1)
        dia_1 = dia_1 + datetime.timedelta(days=1)
        if dia == datetime.datetime(year=2023,month=11,day=29):
            break
    else:
        dia = dia + datetime.timedelta(days=1)
        dia_1 = dia_1 + datetime.timedelta(days=1)
        continue



In [3]:
poc = pd.DataFrame(poc)
poc
poc.to_csv('poc_dias_winfut.csv',index='False')

<h1>Fazendo comparação com o gráfico diário</h1>

In [4]:
data = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_D1, 0, 252))
data['Data'] = pd.to_datetime(data['time'],unit='s')
mesclagem = data.merge(poc,on='Data')
del mesclagem['time']
mesclagem = mesclagem[-252:]

In [6]:
dentro_do_dia = mesclagem.loc[(mesclagem['high'] > mesclagem['POC'].shift(1)) & (mesclagem['low'] < mesclagem['POC'].shift(1))]

<h1>Probabilidade</h1>

In [7]:
total = len(mesclagem['POC'])
favoraveis = len(dentro_do_dia['POC'])
probabilidade = favoraveis/total
print(f'A probabilidade de vir buscar a POC do dia anterior do dolar {ativo} é : {round(probabilidade*100,2)}%.',total,favoraveis)
index_proximo = dentro_do_dia.index  
veio_buscar_a_poc = 0

for k in range(1,len(index_proximo)):
    if index_proximo[k] == index_proximo[k-1]+1:
        veio_buscar_a_poc +=1
    
print(f'Probabilidade de se em um dia ele vier buscar a POC, no outro ele vir buscar:{round(veio_buscar_a_poc*100/len(index_proximo),2)}%')

A probabilidade de vir buscar a POC do dia anterior do dolar WIN$N é : 67.07%. 164 110
Probabilidade de se em um dia ele vier buscar a POC, no outro ele vir buscar:66.36%


# Dias que não vem buscar

In [40]:
fora_do_dia = mesclagem.loc[(mesclagem['high'] < mesclagem['POC'].shift(1)) | (mesclagem['low'] > mesclagem['POC'].shift(1))]

index_proximo = fora_do_dia.index  
veio_buscar_a_poc = 0

for k in range(1,len(index_proximo)):
    if index_proximo[k] != index_proximo[k-1]+1:
        veio_buscar_a_poc +=1
    
print(f'Probabilidade de se em um dia ele não vier buscar a POC, no outro ele vir buscar:{round(veio_buscar_a_poc*100/len(index_proximo),2)}%')

Probabilidade de se em um dia ele não vier buscar a POC, no outro ele vir buscar:66.2%


<h1>DOLPRO</h1>

In [65]:
poc_pro = {'Data':[],'POC':[],'Volume':[]}

dia = datetime.datetime(year=2023,month=1,day=1)
dia_1 = dia + datetime.timedelta(days=1)
for i in range(400):
    ticks_mini = pd.DataFrame(mt5.copy_ticks_range('WDO$N',dia,dia_1,mt5.COPY_TICKS_ALL))
    ticks_cheio = pd.DataFrame(mt5.copy_ticks_range('DOL$N',dia,dia_1,mt5.COPY_TICKS_ALL))
    grupo_mini = ticks_mini.groupby('last').agg({'volume':'sum'}) 
    grupo_cheio = ticks_cheio.groupby('last').agg({'volume':'sum'}) 
    grupo_cheio.rename(columns={'volume':'volume_cheio'},inplace=True)
    fusao = grupo_mini.merge(grupo_cheio,on='last')
    fusao['soma'] = grupo_mini['volume'] +  grupo_cheio['volume_cheio']
    y = tuple((fusao.loc[(fusao['soma'] == fusao['soma'].max())].reset_index()).values)
    if len(y)!=0:
        last = y[0][0]
        volume = y[0][3]
        poc_pro['Data'].append(dia)
        poc_pro['POC'].append(last)
        poc_pro['Volume'].append(volume)
        dia = dia + datetime.timedelta(days=1)
        dia_1 = dia_1 + datetime.timedelta(days=1)
        if dia == datetime.datetime(year=2023,month=10,day=8):
            break
    else:
        dia = dia + datetime.timedelta(days=1)
        dia_1 = dia_1 + datetime.timedelta(days=1)
        continue



In [66]:
poc_pro = pd.DataFrame(poc_pro)
data = pd.DataFrame(mt5.copy_rates_from_pos('WDO$N', mt5.TIMEFRAME_D1, 0, 252))
data['Data'] = pd.to_datetime(data['time'],unit='s')
mesclagem = data.merge(poc_pro,on='Data')
del mesclagem['time']
mesclagem = mesclagem[-22:]
dentro_do_dia = mesclagem.loc[(mesclagem['high'] > mesclagem['POC'].shift(1)) & (mesclagem['low'] < mesclagem['POC'].shift(1))]
total = len(mesclagem['POC'])
favoraveis = len(dentro_do_dia['POC'])
probabilidade = favoraveis/total

print(f'A probabilidade de vir buscar a POC do dia anterior do DOLPRO é : {round(probabilidade*100,2)}%.')

A probabilidade de vir buscar a POC do dia anterior do DOLPRO é : 72.73%.


In [67]:
poc_pro

,Data,POC,Volume
0,2023-01-02,5375.0,49646.0
1,2023-01-03,5405.0,52109.0
2,2023-01-04,5482.0,113126.0
3,2023-01-05,5412.0,58595.0
4,2023-01-06,5269.0,69320.0
...,...,...,...
194,2023-10-04,5170.0,107886.0
195,2023-10-05,5187.0,122381.0
196,2023-10-06,5170.0,81445.0
197,2023-10-09,5180.0,95292.0


In [68]:
poc

,Data,POC,Volume
0,2023-01-02,5380.0,36187.0
1,2023-01-03,5405.0,45494.0
2,2023-01-04,5482.0,101646.0
3,2023-01-05,5412.0,55460.0
4,2023-01-06,5269.0,66680.0
...,...,...,...
137,2023-07-14,4816.0,127428.0
138,2023-07-17,4821.5,40779.0
139,2023-07-18,4824.0,100518.0
140,2023-07-19,4803.0,54924.0
